# Objective

Make an AI Agent that assists TTRPG Game Masters on their campaigns

Tools the AI Agent will use:
- NPC character generator with stats and images
- Dice roller
- Idea generator for planning campaigns
- Generate music playlist for game sessions
- Have a knowledge base available for the campaign's lore and world

In [1]:
# Install libraries
!pip install smolagents -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import login
login()

In [3]:
# Import objects from smolagents
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool

## Tool 1- NPC Character Generator

This stat generator is based off of D&D 5E, updates will be made so that characters are not IP dependant and can be made no matter what the game master is wanting for a character.

In [7]:
# Define NPC character generator with stats
import random
from typing import Dict, Any, List, Optional

def roll_stat():
  """
  Rolls 4d6 and removes the lowest roll.
  Returns:
    The sum of the 3 highest rolls.
  """
  rolls = [random.randint(1,6) for i in range(4)]
  rolls.remove(min(rolls))
  return sum(rolls)

@tool
def generate_npc_character(query: str,
                           races: Optional[List[str]] = None,
                           classes: Optional[List[str]] = None,
                           backstory: Optional[List[str]] = None,) -> Dict[str, Any]:
   """
    Generates a randomized NPC character with stats and customizable attributes.

    Args:
        query: A short description or name for the NPC.
        races: An optional list of race/species names (default is setting-neutral).
        classes: An optional list of class/profession names (default is setting-neutral).
        backstory: An optional custom backstory; if not provided, the agent will generate one.

    Returns:
        A dictionary containing NPC details including stats, race, class, and backstory.
    """
    # Setting default list of races and classes if Game Master does not provide any
   default_races = ["Human", "Elf", "Dwarf", "Gnome", "Halfling"]
   default_classes = ["Barbarian", "Bard", "Cleric", "Druid", "Fighter"]

   npc = {
          "name": query,
          "race": random.choice(races if races else default_races),
          "class": random.choice(classes if classes else default_classes),
          "stats": {
              "strength": roll_stat(),
              "dexterity": roll_stat(),
              "constitution": roll_stat(),
              "intelligence": roll_stat(),
              "wisdom": roll_stat(),
              "charisma": roll_stat()
          },
          "backstory": backstory if backstory else f"{query} has a mysterious past, known only to a few..."
      }

   return npc # Fixed indentation here to align with the function definition

In [8]:
# Test
print(generate_npc_character('Jeery'))

{'name': 'Jeery', 'race': 'Human', 'class': 'Fighter', 'stats': {'strength': 12, 'dexterity': 13, 'constitution': 16, 'intelligence': 14, 'wisdom': 17, 'charisma': 7}, 'backstory': 'Jeery has a mysterious past, known only to a few...'}


In [11]:
print(generate_npc_character('Wagma', ['Unopan'], ['Merc'], 'Brother of the great Allen of Unopa.'))

{'name': 'Wagma', 'race': 'Unopan', 'class': 'Merc', 'stats': {'strength': 13, 'dexterity': 8, 'constitution': 16, 'intelligence': 13, 'wisdom': 16, 'charisma': 11}, 'backstory': 'Brother of the great Allen of Unopa.'}


In [12]:
print(generate_npc_character('Brutis'))

{'name': 'Brutis', 'race': 'Dwarf', 'class': 'Fighter', 'stats': {'strength': 10, 'dexterity': 12, 'constitution': 14, 'intelligence': 11, 'wisdom': 14, 'charisma': 13}, 'backstory': 'Brutis has a mysterious past, known only to a few...'}


## Tool 2- Multi Dice Roller

In [15]:
# Define dice roller function'
import random

@tool
def roll_dice(dice_type: str, amount: int) -> list:
  """
  Rolls different types of dice and returns the results as a list.
  Args:
      dice_type: The type of dice being rolled (e.g., "d6", "d20").
      amount: The number of dice to roll.
  Returns:
      A list of integers representing the dice roll results.
  """
  dice_sides = {
    'd4': 4,
    'd6': 6,
    'd8': 8,
    'd10': 10,
    'd12': 12,
    'd20': 20,
    'd100': 100
  }

  if dice_type not in dice_sides:
    raise ValueError(f"Invalid dice type: {dice_type}. Supported types: {', '.join(dice_sides.keys())}")
  return [random.randint(1, dice_sides[dice_type]) for _ in range(amount)]

In [16]:
# Test
print(roll_dice('d4', 3))

[3, 1, 3]


In [17]:
print(roll_dice('d6', 3))

[2, 6, 4]


In [18]:
print(roll_dice('d20', 10))

[18, 4, 3, 13, 13, 11, 13, 16, 20, 1]


In [19]:
print(roll_dice('d100', 2))

[94, 2]


In [20]:
print(roll_dice('d1', 5))

ValueError: Invalid dice type: d1. Supported types: d4, d6, d8, d10, d12, d20, d100

## Tool 3 - Idea/Campaign Planning Generator

In [24]:
@tool
def campaign_planner(query: str, max_results: int = 5) -> str:
  """
    Generates a structured campaign plan based on a user query.

    Args:
        query: The query to search for campaign ideas.
        max_results: The number of search results to process (default: 5).

    Returns:
        A structured campaign outline or a list of ideas.
    """
  search = DuckDuckGoSearchTool()
  raw_results = search(query)

  # Clean up results, max 5
  results = raw_results.split('\n')[:max_results]
  structured_results = "\n".join(f"- {result}" for result in results if result.strip())

  return f"Here are some campaign ideas based on your search:\n\n{structured_results}"

In [25]:
# Test
print(campaign_planner('Help me plan out my Avatar campaign'))

Here are some campaign ideas based on your search:

- ## Search Results
- [Seven Ways To Leverage Avatars For Effective Marketing Campaigns - Forbes](https://www.forbes.com/councils/forbesbusinesscouncil/2024/09/20/seven-ways-to-leverage-avatars-for-effective-marketing-campaigns/)
- 2. Improve Customer Support. Avatars can be employed in customer support to provide a more interactive and human-like experience. These digital assistants can guide users through product tutorials ...


In [26]:
print(campaign_planner('Help me plan out my Cyberpunk campaign'))

Here are some campaign ideas based on your search:

- ## Search Results
- [What you recommend picking up to run a full campaign for a ... - Reddit](https://www.reddit.com/r/cyberpunkred/comments/16w5vxm/what_you_recommend_picking_up_to_run_a_full/)
- as for a campaign that is fully fleshed out. That's a negative. There are some premade adventures, but not alot. At some point you will need to start building your own adventures and campaign. But the "weekly job" style of campaign makes it easier in some ways. You can have a decent campaign just by picking up a new job every week or two.


In [27]:
print(campaign_planner('Help me plan out my DnD campaign'))

Here are some campaign ideas based on your search:

- ## Search Results
- [DnD 5e Campaign Planning: A Practical Guide - RPGBOT](https://rpgbot.net/dnd5/dungeonmasters/campaign_planning/)
- To illustrate the process, we'll plan out content for taking a party of four characters from 4th level to 5th level. The Skeleton specifies that it will take 13.87 encounters (10 regular and 3 hoard encounters) to take players from 4th level to 5th level, and the daily experience budget for 4th level gives us room for 6.18 encounters per day.


## Tool 4 - Music playlist generator

In [29]:
@tool
def cook_playlist(query: str, mood: str = "ambient") -> str:
    """
    Generates a structured playlist of songs based on a user query and mood.

    Args:
        query: The theme of the campaign (e.g., "dark fantasy," "sci-fi horror").
        mood: The type of music desired (e.g., "epic battle," "tavern music").

    Returns:
        A structured list of music recommendations.
    """
    search = DuckDuckGoSearchTool()
    raw_results = search(f"{query} {mood} TTRPG soundtrack")

    # Extract relevant songs from search results
    results = raw_results.split("\n")[:5]  # Get top 5 results
    structured_results = "\n".join(f"- {result}" for result in results if result.strip())

    return f"Here are some music recommendations for '{query}' ({mood} mood):\n\n{structured_results}"

In [30]:
# Test
print(cook_playlist('Songs for a Cyberpunk campaign'))

Here are some music recommendations for 'Songs for a Cyberpunk campaign' (ambient mood):

- ## Search Results
- [Tabletop Audio - Ambiences and Music for Tabletop Role Playing Games](https://tabletopaudio.com/)
- Original, 10 minute ambiences and music for Tabletop Role Playing Games. Tabletop Audio - Ambiences and Music for Tabletop Role Playing Games Professionally produced, user supported, advertising-free ambient audio.


In [31]:
print(cook_playlist('Songs for a DnD campaign'))

Here are some music recommendations for 'Songs for a DnD campaign' (ambient mood):

- ## Search Results
- [Tabletop Audio - Ambiences and Music for Tabletop Role Playing Games](https://tabletopaudio.com/)
- Original, 10 minute ambiences and music for Tabletop Role Playing Games. Tabletop Audio - Ambiences and Music for Tabletop Role Playing Games Professionally produced, user supported, advertising-free ambient audio.


In [32]:
print(cook_playlist('Songs for a James Cameron Avatar campaign'))

Here are some music recommendations for 'Songs for a James Cameron Avatar campaign' (ambient mood):

- ## Search Results
- [what do you lads do for music? : r/AvatarLegendsTTRPG - Reddit](https://www.reddit.com/r/AvatarLegendsTTRPG/comments/u0eye8/what_do_you_lads_do_for_music/)
- There's a lot of Avatar inspired lofi. Maybe give some a listen and make a list of the different moods so you can pick certain ones for different settings. Here's a playlist. I've been planning a campaign focused on the band that plays secret tunnel. Basically the PCs are escorting them on their reunion tour.


## Tool 5 - Knowledge Base for campaign (Agentic RAG)

In [35]:
@tool
def knowledge_base(document: str) -> str:
  """
  Retrirves campaign lore knowledge based on the document the GM provides.
  Args:
    document: The document to retrieve knowledge from.
  Returns:
    The retrieved knowledge.
  """
  with open(document, 'r') as f:
    return f.read()

In [36]:
# Test
print(knowledge_base('lore.txt'))

FileNotFoundError: [Errno 2] No such file or directory: 'lore.txt'

# Agent Configuration

In [37]:
# Putting everything together
agent = CodeAgent(
    tools=[generate_npc_character, roll_dice, campaign_planner, cook_playlist, knowledge_base],
    model=HfApiModel(),
    max_steps=5,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None
    )

In [40]:
# Run Agent
agent.run('Generate an NPC named Grody that I can use for a Cyberpunk roleplay campaign')

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Generate an NPC named Grody that I can use for a Cyberpunk roleplay campaign                                    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  npc_description = "Grody - A mysterious and elusive hacker in a Cyberpunk setting."                              
  races = ["Human", "Cybernetic", "Synthetic"]                                                                     
  classes = ["Hacker", "Thief"]                                                                                    
  backstory = ["Grody was once a renowned hacker for a major cybercrime syndicate but decided to turn his skills   
  towards something more... legal.", "He often wears eccentric cybernetic enhancements and modified clothing, and  
  is known for his sharp wit and quick reflexes.", "Grody's ultimate goal is to uncover the truth behind the       
  powerful corporations that control the cyberworld."]                                                             
                                                                                                                   
  generated_npc = generate_npc_character(query=npc_description, races=races, classes=classes,                      
  backstory=backstory)                                                                                             
  print(generated_npc)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'name': 'Grody - A mysterious and elusive hacker in a Cyberpunk setting.', 'race': 'Synthetic', 'class': 'Thief', 
'stats': {'strength': 9, 'dexterity': 14, 'constitution': 9, 'intelligence': 15, 'wisdom': 10, 'charisma': 10}, 
'backstory': ['Grody was once a renowned hacker for a major cybercrime syndicate but decided to turn his skills 
towards something more... legal.', 'He often wears eccentric cybernetic enhancements and modified clothing, and is 
known for his sharp wit and quick reflexes.', "Grody's ultimate goal is to uncover the truth behind the powerful 
corporations that control the cyberworld."]}

Out: None

[Step 1: Duration 10.48 seconds| Input tokens: 2,531 | Output tokens: 210]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  npc_name = "Grody - A mysterious and elusive hacker in a Cyberpunk setting."                                     
  npc_race = "Human"                                                                                               
  npc_class = "Thief"                                                                                              
  npc_stats = {                                                                                                    
      "Strength": 16,                                                                                              
      "Dexterity": 11,                                                                                             
      "Constitution": 14,                                                                                          
      "Intelligence": 10,                                                                                          
      "Wisdom": 15,                                                                                                
      "Charisma": 10                                                                                               
  }                                                                                                                
  npc_backstory = [                                                                                                
      "Grody was once a renowned hacker for a major cybercrime syndicate but decided to turn his skills towards    
  something more... legal.",                                                                                       
      "He often wears eccentric cybernetic enhancements and modified clothing, and is known for his sharp wit and  
  quick reflexes.",                                                                                                
      "Grody's ultimate goal is to uncover the truth behind the powerful corporations that control the             
  cyberworld."                                                                                                     
  ]                                                                                                                
                                                                                                                   
  final_npc_info = {                                                                                               
      "Name": npc_name,                                                                                            
      "Race": npc_race,                                                                                            
      "Class": npc_class,                                                                                          
      "Stats": npc_stats,                                                                                          
      "Backstory": npc_backstory                                                                                   
  }                                                                                                                
                                                                                                                   
  final_answer(final_npc_info)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Name': 'Grody - A mysterious and elusive hacker in a Cyberpunk setting.', 'Race': 'Human', 
'Class': 'Thief', 'Stats': {'Strength': 16, 'Dexterity': 11, 'Constitution': 14, 'Intelligence': 10, 'Wisdom': 15, 
'Charisma': 10}, 'Backstory': ['Grody was once a renowned hacker for a major cybercrime syndicate but decided to 
turn his skills towards something more... legal.', 'He often wears eccentric cybernetic enhancements and modified 
clothing, and is known for his sharp wit and quick reflexes.', "Grody's ultimate goal is to uncover the truth 
behind the powerful corporations that control the cyberworld."]}

[Step 4: Duration 18.28 seconds| Input tokens: 13,947 | Output tokens: 1,004]

{'Name': 'Grody - A mysterious and elusive hacker in a Cyberpunk setting.',
 'Race': 'Human',
 'Class': 'Thief',
 'Stats': {'Strength': 16,
  'Dexterity': 11,
  'Constitution': 14,
  'Intelligence': 10,
  'Wisdom': 15,
  'Charisma': 10},
 'Backstory': ['Grody was once a renowned hacker for a major cybercrime syndicate but decided to turn his skills towards something more... legal.',
  'He often wears eccentric cybernetic enhancements and modified clothing, and is known for his sharp wit and quick reflexes.',
  "Grody's ultimate goal is to uncover the truth behind the powerful corporations that control the cyberworld."]}

In [39]:
agent.run('Roll 4 d20 and 3 d10 dice')

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Roll 4 d20 and 3 d10 dice                                                                                       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  d20_rolls = roll_dice(dice_type="d20", amount=4)                                                                 
  d10_rolls = roll_dice(dice_type="d10", amount=3)                                                                 
  print("d20 rolls:", d20_rolls)                                                                                   
  print("d10 rolls:", d10_rolls)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
d20 rolls: [13, 13, 16, 18]
d10 rolls: [5, 8, 9]

Out: None

[Step 1: Duration 5.27 seconds| Input tokens: 2,527 | Output tokens: 113]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({"d20 rolls": [13, 13, 16, 18], "d10 rolls": [5, 8, 9]})                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'d20 rolls': [13, 13, 16, 18], 'd10 rolls': [5, 8, 9]}

[Step 2: Duration 8.00 seconds| Input tokens: 5,335 | Output tokens: 194]

{'d20 rolls': [13, 13, 16, 18], 'd10 rolls': [5, 8, 9]}

In [41]:
agent.run('Help me plan out my Invincible roleplay campaign')

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Help me plan out my Invincible roleplay campaign                                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  campaign_plan = campaign_planner(query="Invincible roleplay campaign")                                           
  print(campaign_plan)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Here are some campaign ideas based on your search:

- ## Search Results
- [Invincible - Superhero Roleplaying by Free League - 
Kickstarter](https://www.kickstarter.com/projects/1192053011/invincible-the-roleplaying-game)
- Free League is raising funds for Invincible - Superhero Roleplaying on Kickstarter! Forge your own superhero 
legend in the official Invincible RPG from Free League and Skybound, based on the groundbreaking comic book.

Out: None

[Step 1: Duration 4.02 seconds| Input tokens: 2,524 | Output tokens: 62]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(campaign_plan)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Setting': "Regal City, a bustling metropolis with advanced technology and a diverse 
population, where a small group of superheroes operates under Supreme Being's watchful eye.", 'Missions': ['Destroy
the Qward enclave in the city.', 'Rescue people during a hostage situation at the Infirmary.', 'Defeat the Crimson 
Pig to prevent it from destroying parts of the city.', 'Protection against the impact of a meteor over Regal 
City.', 'Dealing with a crime spree by the Amalgama and a few other heroes.'], 'Characters': {'Zane': {'Role': 
'Hero', 'Class': 'Superhuman', 'Backstory': 'A street-smart young man transformed overnight into a superhuman by 
Supreme Being in the Infirmary.'}, 'The Crimson Pig': {'Role': 'Villain', 'Class': 'Cyborg', 'Backstory': "Created 
from a crime lord's body, he has a penchant for destroying as much of the city as possible."}}, 'Enemies': ['The 
Crimson Pig', 'The Archenemy', 'The Alien Enforcer', 'Other Supervillains like the Amalgama', "The O'Max 
Corporation"], 'Plot Hooks': ['The appearance of other supervillains leading to a war between heroes and 
villains.', "The hero's discovery of a secret technology that could change the world.", 'The return of an old 
enemy.', 'A major city event where the heroes and the city government need to work together.']}

[Step 4: Duration 4.13 seconds| Input tokens: 12,697 | Output tokens: 734]

{'Setting': "Regal City, a bustling metropolis with advanced technology and a diverse population, where a small group of superheroes operates under Supreme Being's watchful eye.",
 'Missions': ['Destroy the Qward enclave in the city.',
  'Rescue people during a hostage situation at the Infirmary.',
  'Defeat the Crimson Pig to prevent it from destroying parts of the city.',
  'Protection against the impact of a meteor over Regal City.',
  'Dealing with a crime spree by the Amalgama and a few other heroes.'],
 'Characters': {'Zane': {'Role': 'Hero',
   'Class': 'Superhuman',
   'Backstory': 'A street-smart young man transformed overnight into a superhuman by Supreme Being in the Infirmary.'},
  'The Crimson Pig': {'Role': 'Villain',
   'Class': 'Cyborg',
   'Backstory': "Created from a crime lord's body, he has a penchant for destroying as much of the city as possible."}},
 'Enemies': ['The Crimson Pig',
  'The Archenemy',
  'The Alien Enforcer',
  'Other Supervillains like the Amalgama

In [42]:
agent.run('Make a playlist that I can use for a James Cameron Avatar roleplay campaign')

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Make a playlist that I can use for a James Cameron Avatar roleplay campaign                                     │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist = cook_playlist(query="Avatar", mood="epic world music")                                                
  print(playlist)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Here are some music recommendations for 'Avatar' (epic world music mood):

- ## Search Results
- [what do you lads do for music? : r/AvatarLegendsTTRPG - 
Reddit](https://www.reddit.com/r/AvatarLegendsTTRPG/comments/u0eye8/what_do_you_lads_do_for_music/)
- A subreddit for the table-top RPG set in the Avatar Universe. Avatar Legends is developed by Magpie Games using 
the Powered by The Apocalypse System. ... A lot ofthe avatar soundtrack is very grandiose or emotional and doesnt 
seem like itll work all that well if used too much. Im kinda stumped, which id why i turn to you people for some 
...

Out: None

[Step 1: Duration 5.32 seconds| Input tokens: 2,529 | Output tokens: 85]

...

Execution logs:
Here are some music recommendations for 'Ethereal Forest by Kevin MacLeod, Puzzle Solver by Kevin MacLeod, Where 
Wizards Wait by Kevin MacLeod, The Canyon by Kevin MacLeod, The Forest Murmurs by Kevin MacLeod' (epic world music 
mood):

- ## Search Results
- [Crossword Solver - Missing Letters? We Help Finish Your Crossword!](https://www.crosswordsolver.org/)
- The Crossword Solver helps you finish crossword puzzles with missing letter search, clue database, and forum. 
Used by millions in the UK, USA, and Australia!

Out: None

[Step 5: Duration 13.04 seconds| Input tokens: 15,891 | Output tokens: 578]

Reached max steps.

[Step 6: Duration 27.60 seconds| Input tokens: 17,729 | Output tokens: 812]

"It looks like there was a mix-up in the calls. Let's manually create a playlist based on my previous selections of tracks that should fit well with the epic and fantasy themes of the Avatar world. Here is a curated list of tracks that you can use for your Avatar roleplay campaign:\n\n1. **Ethereal Forest by Kevin MacLeod**\n2. **Puzzle Solver by Kevin MacLeod**\n3. **Where Wizards Wait by Kevin MacLeod**\n4. **The Canyon by Kevin MacLeod**\n5. **The Forest Murmurs by Kevin MacLeod**\n\nYou can find these tracks on platforms like YouTube, Spotify, or SoundCloud. They are all licensed under a Creative Commons Attribution license, which means they are free to use as long as you give credit to the creator, Kevin MacLeod, and link back to his website, [Incompetech](https://incompetech.com/).\n\nTo create a playlist:\n\n1. Go to your preferred music platform.\n2. Search for each track by name and artist.\n3. Add the tracks to a new playlist.\n\nThis should give you the perfect background mu

# Gradio Interface

In [44]:
!pip install gradio -U

In [ ]:
import gradio as gr

In [ ]:
def chat_with_agent(message, history):
  return agent(message)

In [ ]:
gr.ChatInterface(
    fn=chat_with_agent,
    chatbot=gr.Chatbot(height=200),
    textbox = gr.Textbox(placeholer='Ask the Game Master AI Agent anything!'),
    title='TTRPG Game Master AI'
).launch()